# Starling Fraud DS Task  

## Intro  
You are the first person with access to this data and as such are tasked with completing some exploratory analysis and data visualisation before creating a model to predict the winner of the race This piece of work represents some adhoc analysis and a first draft model attempt. The idea is to discuss some of of your analysis and output in our discussion so the aim of this exercise is more to facilitate our discussion than to produce a perfect set of results. As such, please do not spend more than 90 minutes in total on these tasks. 

The output of this work will need to be understood by other data scientists and should be useful for any future projects using this data. As such, please provide appropriate annotations/commentary so that the work can be easily followed. 

You are free to use any tools at your disposal for these tasks but it will need to be reproducible by us in order to assess it. Ideally the model would be in the form of a python/R notebook but if you would prefer to use other tools or link to any other open source data, please do so.

## Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from src import PRJPATH

ModuleNotFoundError: No module named 'src'

## Task 1 Data Visualisation:  
(45 Minutes Max)

Consider the best way to visualise this data. 

1) Create some key components that may be used within a data visualisation dashboard to present your analysis to other data scientists and/or a wider audience.  
2) As you explore the data, please note down any questions or concerns that come to mind, along with any insights and conclusions that you are able to draw using the visualisation components that you have built.


In [ ]:
df = pd.read_csv()